<a href="https://colab.research.google.com/github/SSarvesh7206/Capstone_project_S.Sarvesh/blob/main/Model_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pathway bokeh --quiet

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

In [ ]:
data=pd.read_csv("/content/dataset.csv",index_col=0)

In [ ]:
data['Timestamp'] = pd.to_datetime(data['LastUpdatedDate'] + ' ' + data['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')
data = data.sort_values('Timestamp').reset_index(drop=True)

In [ ]:
data["Occupancy_rate"]=data["Occupancy"]/data["Capacity"]

In [ ]:
class ParkingSchema1(pw.Schema):
  Timestamp : str
  Occupancy_rate  : float

In [ ]:
data[["Timestamp","Occupancy_rate"]].to_csv("parking_stream1.csv",index=False)

In [ ]:
data_stream1=pw.demo.replay_csv("/content/parking_stream1.csv",schema=ParkingSchema1,input_rate=1800)

In [ ]:
fmt = "%Y-%m-%d %H:%M:%S"
data_with_time1 = data_stream1.with_columns(
    t = data_stream1.Timestamp.dt.strptime(fmt),
    day = data_stream1.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

In [ ]:
import datetime

delta_window1 = (
    data_with_time1.windowby(
        pw.this.t,
        instance=pw.this.day,
        window=pw.temporal.tumbling(duration=datetime.timedelta(minutes=30)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t=pw.this._pw_window_end,
        occ_rate_avg=pw.reducers.avg(pw.this.Occupancy_rate),
    )
    .with_columns(
    price=pw.if_else(pw.this.occ_rate_avg>0.7,10*(1+(pw.this.occ_rate_avg)),
          pw.if_else(pw.this.occ_rate_avg>0.5,10*(1+0.7*pw.this.occ_rate_avg),
          pw.if_else(pw.this.occ_rate_avg>0.3,10*(1+0.5*pw.this.occ_rate_avg),10
          )))
)
)

In [ ]:
pn.extension()


def price_plotter1(source):

    fig = bokeh.plotting.figure(
        height=400,
        width=1000,
        title="Pathway: Daily Parking Price_Model1",
        x_axis_type="datetime",
    )

    fig.line("t", "price", source=source, line_width=2, color="navy")


    fig.circle("t", "price", source=source, size=6, color="red")

    return fig


viz = delta_window1.plot(price_plotter1, sorting_col="t")



pn.Column(viz).servable()

In [ ]:
%%capture --no-display
pw.run()